In [1]:
DATASET_URI = 'gs://vtx-datasets-public/breast_cancer_data.csv'
PIPELINE_ROOT_PATH = 'gs://vtx-root-path'
PROJECT_ID= 'jchavezar-demo'
MODELS_URI = 'gs://vtx-models/lightgbm'
PRED_IMAGE_URI = 'us-central1-docker.pkg.dev/jchavezar-demo/predictions/pred_lightgbm_cpu:latest'

In [6]:
%%bash
gcloud ai models upload \
  --region=us-central1 \
  --display-name=lightgbm \
  --container-image-uri=us-central1-docker.pkg.dev/jchavezar-demo/predictions/pred_lightgbm_cpu:latest \
  --container-env-vars=PROJECT_ID=jchavezar-demo \
  --container-ports=8080 \
  --container-health-route=/health_check \
  --artifact-uri=gs://vtx-models/lightgbm

Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [2508373394647416832]...
.......................................................................................................................................................................................................................................................................................................................................................................................................done.


In [10]:
from google_cloud_pipeline_components import aiplatform as gcc
from kfp.v2.dsl import pipeline, Condition
from google.cloud import aiplatform
from google_cloud_pipeline_components.types import artifact_types
from google_cloud_pipeline_components import aiplatform as gcc
from google_cloud_pipeline_components.v1.custom_job import CustomTrainingJobOp as custom_job
from google_cloud_pipeline_components.v1.model import ModelUploadOp
from kfp.v2.components import importer_node

@pipeline(name='lightgbm-gpu-1')
def pipeline():
    import_unmanaged_model_op = importer_node.importer(
        artifact_uri="gs://vtx-models/lightgbm",
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": PRED_IMAGE_URI,
                "env": [
                    {
                        "name": "PROJECT_ID",
                        "value": PROJECT_ID},
                ],
                "predictRoute": "/predict",
                "healthRoute": "/health_check",
                "ports": [
                    {
                        "containerPort": 8080
                    }
                ]
            },
        },
    )
    custom_model_upload_job = gcc.ModelUploadOp(
        project=PROJECT_ID,
        display_name="lightgbm-model",
        unmanaged_container_model=import_unmanaged_model_op.outputs["artifact"],
        ).after(import_unmanaged_model_op)

    endpoint_create_job = gcc.EndpointCreateOp(
        project=PROJECT_ID,
        display_name="pipelines-created-endpoint",
    )
        
    custom_model_deploy_job = (gcc.ModelDeployOp(
        model=custom_model_upload_job.outputs["model"],
        endpoint=endpoint_create_job.outputs["endpoint"],
        deployed_model_display_name="lightgbm_model_end",
        traffic_split={"0":"100"},
        dedicated_resources_machine_type="n1-standard-2",
        dedicated_resources_min_replica_count=1,
        dedicated_resources_max_replica_count=1
    )).set_caching_options(False)

In [11]:
from kfp.v2 import compiler
import warnings
warnings.filterwarnings('ignore')

compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='light_cpu.json')

In [12]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="light_cpu",
    template_path="light_cpu.json",
    pipeline_root=PIPELINE_ROOT_PATH,
    parameter_values={
    },
    enable_caching=False
)

job.submit(service_account='vtx-pipe@jchavezar-demo.iam.gserviceaccount.com')

Creating PipelineJob
PipelineJob created. Resource name: projects/569083142710/locations/us-central1/pipelineJobs/lightgbm-gpu-1-20220814230617
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/569083142710/locations/us-central1/pipelineJobs/lightgbm-gpu-1-20220814230617')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/lightgbm-gpu-1-20220814230617?project=569083142710
